In [12]:
from acdesign.airfoils.polar import UIUCPolar

import numpy as np
import pandas as pd

afoil = UIUCPolar.local("CLARKYB")

# afoil.drag
# afoil.lookup(re=100000.0, cl=2.2)

Fitting a 4th order polynomial for cl=>cd each re, then linearly interpolating between res.

In [13]:
import plotly.express as px
import plotly.graph_objects as go
from functools import partial

polys = {}
gb = afoil.drag.groupby("re")
for re, grp in gb:
    polys[re] = partial(np.polyval, np.polyfit(grp.Cl, grp.Cd, deg=4))

cls = np.linspace(-0.5, 1.5, 100)

fig = go.Figure()

for re in afoil.drag.re.unique():
    cds = polys[re](cls)

    fig.add_trace(
        go.Scatter3d(
            x=cls,
            y=np.full(len(cls), re),
            z=cds,
            mode="lines",
            showlegend=False,
            line=dict(color="red"),
        )
    )
    grp = gb.get_group(re)
    fig.add_trace(
        go.Scatter3d(
            x=grp.Cl,
            y=grp.re,
            z=grp.Cd,
            mode="markers",
            showlegend=False,
            marker=dict(size=3, color="black"),
        )
    )

for cl in np.linspace(-0.5, 1.5, 10):
    res = np.array(list(polys.keys()))

    cds = np.array([polys[re](cl) for re in res])

    fig.add_trace(
        go.Scatter3d(
            x=np.full(len(res), cl),
            y=res,
            z=cds,
            mode="lines",
            showlegend=False,
            line=dict(color="green"),
        )
    )

fig.update_layout(
    template="plotly_white",
    margin=dict(t=40, b=30, l=0, r=0),
    scene=dict(xaxis=dict(title="Cl"),
    yaxis=dict(title="Re"),
    zaxis=dict(title="Cd"),),
    
    width=600,
    height=400,
    title=afoil.name,
).show()

In [14]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(
    go.Scatter3d(
        x=afoil.drag["Cl"],
        y=afoil.drag["re"],
        z=afoil.drag["Cd"],
        mode="markers",
        marker=dict(size=4, color="black"),
        name="data",
    )
)

sweep = afoil.lookup(re="sweep", cl_or_alpha="sweep")

fig.add_trace(
    go.Scatter3d(
        x=sweep.cl,
        y=sweep.re,
        z=sweep.Cd,
        mode="markers",
        marker=dict(size=1),
        name="interpolation",
        showlegend=False,
    )
)


fig.update_layout(
    height=600,
    width=900,
    margin=dict(l=0, r=0, t=30, b=0),
    scene=dict(
        xaxis=dict(title="Cl"),
        yaxis=dict(title="Re"),
        zaxis=dict(title="Cd"),
    ),
)


In [15]:
afoil.pslift

fig = go.Figure()


for re, df in afoil.pslift.groupby("re"):
    fig.add_trace(go.Scatter3d(x=df.Cl, y=df.re, z=df.Cm, name=re, mode="lines"))


sweep = afoil.lookup(re="sweep", cl_or_alpha="sweep")

fig.add_trace(
    go.Scatter3d(
        x=sweep.cl,
        y=sweep.re,
        z=sweep.Cm,
        mode="markers",
        marker=dict(size=1),
        name="interpolation",
        showlegend=False,
    )
)


fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
    width=600,
    height=400,
    xaxis=dict(title="alpha"),
    yaxis=dict(title="Cl"),
)


In [16]:
zvals

NameError: name 'zvals' is not defined